## 设置

In [ ]:
baiduAccessTokenGetUrl = "https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={your own client_id}&client_secret={your own client_secret}"

In [ ]:
# 通用库
import os, datetime, re
import numpy as np
import matplotlib.pyplot as plt

# PDF及图片处理
import fitz
from PIL import Image

# 调百度OCR的API
import requests
import base64

# 绘图
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
def findPDFFiles():
    pdfFiles = []
    for root, dirs, files in os.walk("./pdfs"):
        for file in files:
            if file.endswith(".pdf"):
                pdfFiles.append(file)
    return pdfFiles

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def countContinuousOne(array):
    count = 0
    countList = []
    position = []
    for index in range(len(array)):
        i = array[index]
        if i == 1:
            count += 1
        else:
            if count != 0:
                countList.append(count)
                position.append(index)
                count = 0
    return countList, position

def findCloseTo101(array):
    lineList = []
    for i in range(len(array)):
        if array[i]>100 and array[i]<140:
            lineList.append(i)
    return lineList

def pdf_image(pdfPath, zoom_x, zoom_y, rotation_angle):
    pdf = fitz.open("./pdfs/"+pdfPath)
    for page in pdf.pages():
        mat = fitz.Matrix(zoom_x, zoom_y)
        pix = page.get_pixmap(matrix=mat, dpi=None, colorspace='rgb', alpha=False)

        pix.save("images.png")
        break
    pdf.close()
    
pdfFiles = findPDFFiles()
for fileName in pdfFiles:
    pdf_image(fileName, 10, 10, 0)
    img = Image.open("images.png")
    width, height = img.size
    imgCut = img.crop((0, 0, width/2, 7300))
    imgCutArray = np.array(imgCut)
    imgCutArrayGray = rgb2gray(imgCutArray)
    imgCutArrayGraySum = np.sum(imgCutArrayGray, axis=1)

    emptyLine = np.zeros_like(imgCutArrayGraySum)
    emptyLine[imgCutArrayGraySum>780000] = 1
    countList,position = countContinuousOne(emptyLine)
    lineList = findCloseTo101(countList)
    lineListPosition = [position[i]-50 for i in lineList]
    print(lineListPosition[-1])
    imgCut2 = img.crop((0, lineListPosition[-1], width/2, lineListPosition[-1]+500))
    imgCut2.save(f"images/{fileName[:-4]}.png")

In [ ]:
def findPngFiles(rootDir):
    pngFiles = []
    for root, dirs, files in os.walk(rootDir):
        for file in files:
            if file.endswith(".png"):
                pngFiles.append(file)
    return pngFiles
pngFiles = findPngFiles("./images")

In [ ]:
payload = ""
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}
response = requests.request("POST", baiduAccessTokenGetUrl, headers=headers, data=payload)
access_token = response.json()['access_token']
print(response.json()['access_token'])

In [ ]:
request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/accurate_basic"
textList = []
for pngFile in pngFiles:
    f = open(f"./images/{pngFile}", "rb")
    img = base64.b64encode(f.read())

    params = {"image":img}
    request_url = request_url + "?access_token=" + access_token
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    response = requests.post(request_url, data=params, headers=headers)
    if response:
        # print (response.json())
        text = " ".join([item['words'] for item in response.json()['words_result']])
        print(text)
        textList.append(text)

In [ ]:
srcTextList = textList
textList = [item for item in textList if "Manuscript" in item[:40]]

In [ ]:

textItem = textList[0]
print(textItem)
textItemInforList = re.split('\.|；', textItem)
textItemInforList

In [ ]:

def textDate2Date(text):
    text = text.replace("th", "").replace("st", "").replace("rd", "").replace("nd", "")
    text = text.replace("January", "01").replace("February", "02").replace("March", "03").replace("April", "04")
    text = text.replace("May", "05").replace("June", "06").replace("July", "07").replace("Augu", "08").replace("September", "09")
    text = text.replace("October", "10").replace("November", "11").replace("December", "12")
    text = text.replace(" ", "-")
    return datetime.datetime.strptime(text, "%d-%m-%Y")

In [ ]:
acceptList = []
for textItem in textList:
    print(textItem)
    textItemInforList = re.split('\.|；', textItem)

    textDict = {}
    for item in textItemInforList:
        if "received" in  item:
            textDict["received"] = textDate2Date(" ".join(item.split(" ")[-3:]))
        if "accepted" in  item:
            textDict["accepted"] = textDate2Date(" ".join(item.split(" ")[-3:]))
        if "revised" in item:
            textDict["revised"] = textDate2Date(" ".join(item.split(" ")[-3:]))
        if "Date of publication" in item:
            textDict["publication"] = textDate2Date(" ".join(item.split(" ")[-3:]))
        if "current version" in item:
            textDict["currentVersion"] = textDate2Date(" ".join(item.split(" ")[-3:]))
    acceptList.append(textDict)


In [ ]:
for accept in acceptList:
    accept['received-accepted'] = accept['accepted'] - accept['received']
    if 'revised' not in accept:
        accept['received-revised'] = None
    else:
        accept['received-revised'] = accept['revised'] - accept['received']
    if 'revised' not in accept:
        accept['revised-accepted'] = None
    else:
        accept['revised-accepted'] = accept['accepted'] - accept['revised']


In [ ]:
# hist received
def hist(dataName,dataList = []):
    if dataList == []:
        dataList = [item[dataName] for item in acceptList if dataName in item]
        dataList.sort()

    fig, ax = plt.subplots()
    ax.hist(dataList, bins=30, color='lightblue')
    # title
    ax.set_title(dataName)
hist('received')
hist('accepted')
hist('revised')
hist('publication')

# hist received-accepted
dataList = [item['received-accepted'].days for item in acceptList]
hist('received-accepted', dataList)

# hist accepted-revised
dataList = [item['received-revised'].days for item in acceptList if item['received-revised'] != None]
hist('received-revised', dataList)

# hist revised-accepted
dataList = [item['revised-accepted'].days for item in acceptList if item['revised-accepted'] != None]
hist('revised-accepted', dataList)